# Squashed Entanglement
$$
 \newcommand{\ul}[1]{\underline{#1}}
 \newcommand{\rvalp}[0]{{\ul{\alpha}}}
\newcommand{\alp}[0]{{\alpha}}
 \newcommand{\rvx}[0]{{\ul{x}}}
\newcommand{\rvy}[0]{{\ul{y}}}
$$

The purpose of this notebook is to show how to use entanglish to
calculate the squashed entanglement of a mixed state (either pure or not pure).

Consider a bipartite system
consisting of two parts labelled
by the random variables $\rvx$ and $\rvy$,
and
described by a density matrix $\rho_{\rvx, \rvy}$.
The squashed entanglement of  such a system
is defined as

$$
E_{\rvx, \rvy}(\rho_{\rvx, \rvy}) =
\frac{1}{2}
\min S(\rvx : \rvy|\rvalp)
\;.
$$
The min()---or infimum()
if one wishes to be more mathematically
precise--is
over all density matrices $\rho_{\rvx, \rvy,\rvalp}$
such that ${\rm tr}_\rvalp \; \rho_{\rvx, \rvy,\rvalp}=
\rho_{\rvx, \rvy}$ with $\rho_{\rvx, \rvy}$ held fixed.
If $\rho_{\rvx, \rvy}$ is a pure state, then
$E_{\rvx, \rvy} = S(\rvx) = S(\rvy)$.
Entanglish-Original-Ref discusses other interesting
properties of squashed entanglement

Entanglish-Original-Ref also describes the algo
used by Entanglish to calculate squashed entanglement. The
algorithm is recursive. 
The number of recursive steps 
can be chosen by the user and is called num_ab_steps (ab stands
for Arimoto-Blahut).
Another parameter of the algorithm is num_hidden_states, which is the number of possible 
$\rvalp$ values.

 
**Entanglish-Original-Ref**
* "A New  Algorithm for Calculating
Squashed Entanglement and a Python Implementation Thereof", by R.R.Tucci

First change your working directory to the entanglish directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

C:\Users\rrtuc\Desktop\backed-up\python-projects\entanglish\entanglish\jupyter_notebooks
C:\Users\rrtuc\Desktop\backed-up\python-projects\entanglish


In [2]:
from entanglish.SymNupState import *
from entanglish.SquashedEnt import *

## pure states (symmetrized n-up states)
Next we construct a symmetrized n-up pure state.
Then we compare the Arimoto-Blahut algo entanglement value to the known 
entanglement value, for various possible
bi-partitions of the set of row axes. As expected, they are equal.

recursion_init is a str that specifies how to
initiate the recursion for Kxy_a. There are currently 2 options for recursion_init, 'eigen' and 'eigen+'. 'eigen+' uses Dxy.num_rows^2 degrees of freedom, which is sufficient but 
may not be necessary. 'eigen' uses only Dxy.num_rows degrees of freedom.
With both 'eigen' and 'eigen+', the recursion converges instantly for pure states. 

In [3]:
num_qbits = 4
num_up = 1
dm1 = DenMat(1 << num_qbits, tuple([2]*num_qbits))
st = SymNupState(num_up, num_qbits)
st_vec = st.get_st_vec()
dm1.set_arr_from_st_vec(st_vec)

recursion_init = 'eigen+'
num_ab_steps = 5
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
ecase = SquashedEnt(dm1, num_ab_steps,
    recursion_init=recursion_init, verbose=True)
print('entang_023: algo value, known value\n',
      ecase.get_entang({0, 2, 3}),
      st.get_known_entang(3))
print('entang_02: algo value, known value\n',
      ecase.get_entang({0, 2}),
      st.get_known_entang(2))
print('entang_1: algo value, known value\n',
      ecase.get_entang({1}),
      st.get_known_entang(1))

recursion_init= eigen+
num_ab_steps= 5

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 1.9364916731037083
--ab step= 0 , entang= 1.5791314317444571 , err= 6.694455660188682
--ab step= 1 , entang= 0.5623351446188078 , err= 3.6552010080612974e-14
--ab step= 2 , entang= 0.5623351446188084 , err= 3.0598107196682193e-14
--ab step= 3 , entang= 0.5623351446188087 , err= 3.005109874201847e-14
--ab step= 4 , entang= 0.5623351446188085 , err= 3.0009457033623504e-14
entang_023: algo value, known value
 0.5623351446188085 0.5623351446188083

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 1.9364916731037083
--ab step= 0 , entang= 1.7326078859934175 , err= 7.468056786574437
--ab step= 1 , entang= 0.6931471805599447 , err= 5.829159223514377e-14
--ab step= 2 , entang= 0.6931471805599451 , err= 4.6732109099962156e-14
--ab step= 3 , entang= 0.6931471805599444 , err= 4.627256166575864e-14
--ab step= 4 , entang= 0.6931471805599454 , err= 4.622380825544515e-14
entang_02: algo value, known value
 

## random density matrices
Next we consider 2 random density matrices (actually,
only their eigenvectors are random. Their eigenvalues are specified by the user.)
For each of those 2 density matrices, we calculate the Arimoto-Blahut algo entanglement value, for various possible
bi-partitions of the set of row axes.

In [4]:
np.random.seed(123)
dm = DenMat(8, (2, 2, 2))
evas_of_dm_list = [
    np.array([.07, .03, .25, .15, .3, .1, .06, .04])
    , np.array([.05, .05, .2, .2, .3, .1, .06, .04])
    ]

recursion_init = 'eigen+'
num_ab_steps = 100
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
for evas_of_dm in evas_of_dm_list:
    evas_of_dm /= np.sum(evas_of_dm)
    print('***************new dm')
    print('evas_of_dm\n', evas_of_dm)
    dm.set_arr_to_rand_den_mat(evas_of_dm)
    ecase = SquashedEnt(dm, num_ab_steps,
        recursion_init=recursion_init, verbose=True)
    print('ent_02_1=', ecase.get_entang({0, 2}))

recursion_init= eigen+
num_ab_steps= 100
***************new dm
evas_of_dm
 [0.07 0.03 0.25 0.15 0.3  0.1  0.06 0.04]

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 4.880055036277153e-16
--ab step= 0 , entang= 0.4468814213843033 , err= 9.883987693251486
--ab step= 1 , entang= 0.09985936311777065 , err= 5.244905691396971
--ab step= 2 , entang= 0.0754124292865664 , err= 1.1719234031966845
--ab step= 3 , entang= 0.06127552394088655 , err= 0.7035712210399119
--ab step= 4 , entang= 0.05398144028154518 , err= 0.5574150712016113
--ab step= 5 , entang= 0.04783797593640076 , err= 0.5087942230284682
--ab step= 6 , entang= 0.04233276288542726 , err= 0.4847727513598439
--ab step= 7 , entang= 0.0372839122361038 , err= 0.46763930946688054
--ab step= 8 , entang= 0.032670961183028244 , err= 0.44819037457766586
--ab step= 9 , entang= 0.028509501012267655 , err= 0.42387655470923075
--ab step= 10 , entang= 0.024794809913998586 , err= 0.3975186551910077
--ab step= 11 , entang= 0.021496439519322126 , 

--ab step= 10 , entang= 0.02219054529365812 , err= 0.49543086530036934
--ab step= 11 , entang= 0.018159311023389035 , err= 0.45706408144407157
--ab step= 12 , entang= 0.014930184062287567 , err= 0.41869568572470783
--ab step= 13 , entang= 0.012364445966150632 , err= 0.3857813459588884
--ab step= 14 , entang= 0.010313848016958956 , err= 0.355490103131683
--ab step= 15 , entang= 0.008670446954503374 , err= 0.3302803532692201
--ab step= 16 , entang= 0.00734838354591422 , err= 0.307099514314666
--ab step= 17 , entang= 0.006279469771434276 , err= 0.28811470236600345
--ab step= 18 , entang= 0.005409984773743741 , err= 0.2681893907987783
--ab step= 19 , entang= 0.004699728150420965 , err= 0.2492078434808317
--ab step= 20 , entang= 0.004118986850335815 , err= 0.23167624286762492
--ab step= 21 , entang= 0.0036422711949793294 , err= 0.21312678187262116
--ab step= 22 , entang= 0.003246878653584018 , err= 0.19637045499966885
--ab step= 23 , entang= 0.0029152342079424775 , err= 0.17994753009681572
